<a href="https://colab.research.google.com/github/aarush6848ddh/nba-data-analysis/blob/main/NBA_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)



In [63]:
data = pd.read_excel('nba_players_data.xlsx')
data.drop(columns=['RANK','EFF'], inplace=True)
data['season_start_year'] = data['Year'].str[:4].astype(int)
data['TEAM'].replace(to_replace=['NOP','NOH'], value='NO', inplace=True)
data['Season Type'].replace('Regular%20Season','RS', inplace=True)
rs_df = data[data['Season Type']=='RS']
playoffs_df = data[data['Season Type']=='Playoffs']
data.columns

Index(['Year', 'Season Type', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'GP',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PTS',
       'season_start_year'],
      dtype='object')

In [ ]:
total_cols = ['MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
              'OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS']

In [61]:
data_per_min = data.groupby(['PLAYER','PLAYER_ID','Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col]/data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']

data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)

# Select only numeric columns
numeric_data = data_per_min.select_dtypes(include=['float64', 'int64'])


fig = px.imshow(numeric_data.corr())
fig.update_layout(title="Correlation Matrix of NBA Player Statistics",  title_x=0.48)
fig.show()

In [43]:
fig = px.histogram(x=playoffs_df['MIN'], histnorm='percent')
fig.show()

In [47]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
change_df['POSS_est'] = change_df['FGA']-change_df['OREB']+change_df['TOV']+0.44*change_df['FTA']
change_df = change_df[list(change_df.columns[0:2])+['POSS_est']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['FTA/FGA'] = change_df['FTA']/change_df['FGA']
change_df['TRU%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df


,season_start_year,MIN,POSS_est,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,8707.1,3375.680,1255.0,2886.3,194.4,571.2,675.2,904.5,427.5,1101.1,1528.3,754.0,286.3,182.0,518.9,3377.0,0.434813,0.340336,0.746490,0.600797,0.197900,1.170010,0.154900,0.313377,0.509207,1.453074
1,2004,8414.4,3320.396,1280.5,2844.0,209.6,590.3,721.8,953.4,414.7,1020.6,1435.4,729.3,253.7,174.6,471.6,3491.9,0.450246,0.355074,0.757080,0.569543,0.207560,1.227813,0.163686,0.335232,0.529579,1.546438
2,2005,8675.1,3431.412,1318.9,2882.2,221.2,615.3,753.6,987.3,371.3,1049.2,1419.2,733.6,240.7,148.4,486.1,3610.8,0.457602,0.359499,0.763294,0.556221,0.213483,1.252793,0.167716,0.342551,0.538738,1.509154
3,2006,8730.5,3470.048,1324.0,2913.6,233.5,650.6,727.2,969.2,391.3,1086.0,1476.8,753.6,259.1,164.8,521.3,3607.0,0.454421,0.358899,0.750310,0.569184,0.223298,1.237987,0.176360,0.332647,0.534534,1.445617
4,2007,9111.3,3576.804,1404.4,3086.8,244.6,687.0,758.3,999.1,434.2,1162.9,1596.7,782.7,262.6,183.6,484.6,3810.8,0.454970,0.356041,0.758983,0.557320,0.222561,1.234547,0.174167,0.323669,0.535018,1.615147
5,2008,8648.3,3404.900,1324.7,2889.7,246.3,672.3,715.6,922.5,362.9,1067.1,1427.9,739.8,254.4,161.4,472.2,3611.8,0.458421,0.366354,0.775718,0.558466,0.232654,1.249888,0.185929,0.319237,0.542657,1.566709
6,2009,8832.6,3531.968,1380.8,3006.2,239.2,672.8,739.2,964.7,384.8,1125.0,1508.4,765.3,256.7,176.2,486.1,3736.8,0.459317,0.355529,0.766249,0.554244,0.223804,1.243031,0.173233,0.320903,0.539309,1.574367
7,2010,8742.2,3460.820,1338.9,2950.0,229.2,644.7,728.0,945.5,389.5,1095.9,1484.7,755.0,262.8,178.1,484.3,3632.6,0.453864,0.355514,0.769963,0.563896,0.218542,1.231390,0.171185,0.320508,0.534345,1.558951
8,2011,8301.9,3258.620,1260.4,2838.6,207.8,606.2,618.1,818.0,403.8,1078.0,1480.5,678.8,256.9,184.9,463.9,3343.8,0.444022,0.342791,0.755623,0.538559,0.213556,1.177975,0.164868,0.288170,0.518073,1.463246
9,2012,8404.6,3348.172,1298.8,2874.7,253.5,710.2,626.2,831.3,377.2,1080.8,1456.9,737.0,260.6,174.4,484.9,3476.4,0.451804,0.356942,0.753278,0.567447,0.247052,1.209309,0.195180,0.289178,0.531630,1.519901


In [48]:
change_per48_df = change_df.copy()
for col in change_per48_df.columns[2:18]:
    change_per48_df[col] = (change_per48_df[col]/change_per48_df['MIN'])*48*5

change_per48_df.drop(columns='MIN', inplace=True)

fig = go.Figure()
for col in change_per48_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'],
                             y=change_per48_df[col], name=col))
fig.show()

In [54]:
change_per100_df = change_df.copy()

for col in change_per100_df.columns[3:18]:
    change_per100_df[col] = (change_per100_df[col]/change_per100_df['POSS_est'])*100

change_per100_df.drop(columns=['MIN','POSS_est'], inplace=True)
change_per100_df

fig = go.Figure()
for col in change_per100_df.columns[1:]:
    fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'],
                             y=change_per100_df[col], name=col))
fig.show()

In [50]:
rs_change_df = rs_df.groupby('season_start_year')[total_cols].sum().reset_index()
playoffs_change_df = playoffs_df.groupby('season_start_year')[total_cols].sum().reset_index()

for i in [rs_change_df,playoffs_change_df]:
    i['POSS_est'] = i['FGA']-i['OREB']+i['TOV']+0.44*i['FTA']
    i['POSS_per_48'] = (i['POSS_est']/i['MIN'])*48*5

    i['FG%'] = i['FGM']/i['FGA']
    i['3PT%'] = i['FG3M']/i['FG3A']
    i['FT%'] = i['FTM']/i['FTA']
    i['AST%'] = i['AST']/i['FGM']
    i['FG3A%'] = i['FG3A']/i['FGA']
    i['PTS/FGA'] = i['PTS']/i['FGA']
    i['FG3M/FGM'] = i['FG3M']/i['FGM']
    i['FTA/FGA'] = i['FTA']/i['FGA']
    i['TRU%'] = 0.5*i['PTS']/(i['FGA']+0.475*i['FTA'])
    i['AST_TOV'] = i['AST']/i['TOV']
    for col in total_cols:
        i[col] = 100*i[col]/i['POSS_est']
    i.drop(columns=['MIN','POSS_est'], inplace=True)

rs_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,38.239948,85.797652,5.893882,16.772964,20.196838,26.778507,12.611100,32.280802,44.906963,23.083534,8.383972,5.547478,15.030905,102.580657,94.246321,0.445699,0.351392,0.754218,0.603650,0.195494,1.195611,0.154129,0.312112,0.520622,1.535738
1,2004,38.705771,85.476948,6.224223,17.271325,21.766882,28.543460,12.427989,30.676164,43.119497,23.050596,7.804573,5.012111,14.391918,105.397532,95.343850,0.452821,0.360379,0.762587,0.595534,0.202058,1.233052,0.160809,0.333932,0.532122,1.601635
2,2005,38.540236,84.403409,6.583835,18.005866,21.472834,28.119882,10.726692,30.147548,40.815890,22.299460,7.337524,4.293594,13.950535,105.132280,96.480019,0.456619,0.365649,0.763617,0.578602,0.213331,1.245593,0.170830,0.333161,0.537704,1.598466
3,2006,38.702082,83.711097,6.624725,18.148323,21.164896,27.853178,11.020022,30.630185,41.630650,22.636513,7.475427,4.717977,15.053526,105.228008,96.979004,0.462329,0.365032,0.759874,0.584891,0.216797,1.257038,0.171172,0.332730,0.542741,1.503735
4,2007,39.955332,86.219878,7.125647,19.358538,20.537830,26.865187,11.720352,32.357968,44.069248,22.302233,7.606436,4.948492,13.679791,107.574141,96.149673,0.463412,0.368088,0.764477,0.558179,0.224525,1.247672,0.178340,0.311589,0.543409,1.630305
5,2008,39.582198,85.190768,7.236566,19.289311,20.927501,26.825966,10.911424,31.046887,41.884518,22.550931,7.487460,4.580043,13.917231,107.372738,96.912133,0.464630,0.375159,0.780121,0.569724,0.226425,1.260380,0.182824,0.314893,0.548194,1.620361
6,2009,39.730552,85.383816,6.631935,18.444482,20.036710,26.034572,10.788637,31.680172,42.407749,22.056351,7.374036,4.790775,13.949609,106.059297,98.241224,0.465317,0.359562,0.769619,0.555148,0.216018,1.242148,0.166923,0.304912,0.542501,1.581145
7,2010,39.762870,85.252705,6.651906,18.244817,20.473301,26.382379,10.821328,31.222743,42.015317,22.816804,7.476206,4.768477,13.960376,106.583853,98.170783,0.466412,0.364592,0.776022,0.573822,0.214009,1.250211,0.167289,0.309461,0.544995,1.634398
8,2011,39.885983,87.344198,6.377165,18.004271,18.823596,24.778052,12.556022,33.013049,45.584727,21.526846,7.979284,5.536966,14.309481,104.951853,95.663119,0.456653,0.354203,0.759688,0.539710,0.206130,1.201589,0.159885,0.283683,0.529451,1.504376
9,2012,39.736744,86.658749,7.610868,20.739616,18.070805,23.788971,11.591553,32.090826,43.662350,22.992834,8.041483,5.262515,14.465657,105.145147,96.911494,0.458543,0.366972,0.759630,0.578629,0.239325,1.213324,0.191532,0.274513,0.536682,1.589477


In [52]:
comp_change_df = round(100*(playoffs_change_df-rs_change_df)/rs_change_df,3)
comp_change_df['season_start_year'] = list(range(2003,2023))
comp_change_df

,season_start_year,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,POSS_per_48,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,FTA/FGA,TRU%,AST_TOV
0,2003,-6.776,-0.838,-5.589,2.154,-2.355,0.147,1.026,2.553,1.993,-7.897,2.831,-6.859,5.531,-6.044,-3.050,-5.988,-7.580,-2.498,-1.202,3.017,-5.250,1.273,0.993,-5.371,-12.725
1,2004,-0.887,0.499,3.450,7.135,-0.319,1.448,1.203,0.485,0.622,-11.463,-5.108,11.952,-3.191,-0.536,-1.611,-1.379,-3.440,-1.742,-10.671,6.603,-1.030,4.375,0.944,-1.158,-8.545
2,2005,-0.675,-1.209,-5.213,-1.032,5.683,5.792,2.185,3.550,3.321,-10.303,-10.984,1.811,3.858,0.227,-3.912,0.541,-4.224,-0.102,-9.693,0.179,1.453,-4.569,7.086,0.480,-13.635
3,2006,-3.442,0.737,3.834,8.062,-2.398,0.676,5.669,5.298,5.428,-9.891,-0.283,1.613,-0.496,-2.966,-3.897,-4.148,-3.912,-3.054,-6.678,7.272,-3.675,7.536,-0.060,-3.667,-9.441
4,2007,-4.510,0.244,-10.505,-2.039,8.411,10.359,9.319,1.243,3.379,-4.904,-9.071,9.724,-2.504,-2.501,-5.078,-4.742,-8.641,-1.765,-0.413,-2.278,-2.738,-6.278,10.091,-3.987,-2.462
5,2008,-4.241,-0.938,-0.099,5.863,1.058,2.473,-5.759,2.344,0.309,-9.061,-0.526,8.679,-0.873,-2.996,-5.981,-3.335,-5.631,-1.380,-5.033,6.865,-2.077,4.326,3.443,-2.514,-8.259
6,2009,-4.031,-0.796,5.334,8.250,11.210,12.367,2.477,1.365,1.777,-4.435,-3.624,10.402,-3.370,-0.617,-5.621,-3.262,-2.694,-1.030,-0.421,9.118,0.180,9.758,13.268,-1.474,-1.102
7,2010,-6.812,-0.038,-1.105,5.297,6.915,8.951,10.082,3.575,5.304,-11.050,3.954,19.948,0.603,-3.829,-7.730,-6.777,-6.080,-1.869,-4.548,5.336,-3.792,6.124,8.993,-4.888,-11.583
8,2011,-7.346,-0.650,-0.009,8.072,1.864,3.180,-3.176,0.503,-0.806,-7.848,-2.908,6.016,-1.245,-5.407,-3.625,-6.740,-7.477,-1.275,-0.542,8.778,-4.789,7.919,3.855,-5.223,-6.686
9,2012,-5.896,-2.288,-1.289,5.640,8.666,10.829,-6.963,1.463,-0.846,-10.572,-7.955,-2.529,0.289,-3.100,-3.264,-3.693,-6.558,-1.951,-4.968,8.113,-0.832,4.896,13.424,-2.344,-10.829


In [55]:
fig = go.Figure()
for col in comp_change_df.columns[1:]:
    fig.add_trace(go.Scatter(x=comp_change_df['season_start_year'],
                             y=comp_change_df[col], name=col))
fig.show()

In [68]:
# Filter data for LeBron James
lebron_data = data[data['PLAYER'] == 'LeBron James']

# Plot LeBron James' performance over time
fig = go.Figure()

# Adding metrics as traces
metrics = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA']
for metric in metrics:
    fig.add_trace(go.Scatter(x=lebron_data['season_start_year'], y=lebron_data[metric], mode='lines+markers', name=metric))

# Update layout
fig.update_layout(
    title="LeBron James' Performance Over Time",
    xaxis_title="Season Start Year",
    yaxis_title="Value",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()